# Polyelectrolyte with proton exchange


In this example, a single polyelectrolyte chain is simulated with proton exchange with an implicit solvent.
See https://doi.org/10.1063/1.4757284.

In [9]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/monte_carlo/tutorial/')
import analyze

def chain_config():
    config = feasst.Configuration(feasst.args({
        'cubic_box_length': '20',
        'particle_type0': feasst.install_dir() + '/plugin/chain/forcefield/data.chain10titratable',
        'particle_type1': feasst.install_dir() + '/forcefield/data.lj',
    }))
    config.add_particle_of_type(0)
    return config

class TestChain2Titration(unittest.TestCase):
    """Test a single polyelectrolyte simulation with proton exchange with implicit solvent"""
    def test(self):
        monte_carlo = feasst.MonteCarlo()
        monte_carlo.set(feasst.MakeRandomMT19937(feasst.args({'seed': '1234'})))
        monte_carlo.add(chain_config())
        monte_carlo.add(feasst.Potential(feasst.MakeLennardJones()))
        monte_carlo.add(feasst.Potential(feasst.MakeLennardJones(),
                                         feasst.MakeVisitModelIntra(feasst.args({"cutoff": "1"}))))
        monte_carlo.set(feasst.MakeMetropolis(feasst.args({
            "beta": "1.",
            "chemical_potential0": "1.",
            "chemical_potential1": "-5",
            "pH": "1"
        })))
        
        # trials for chains
        monte_carlo.add(feasst.MakeTrialTranslate(feasst.args({"weight": "1.", "particle_type": "0", "tunable_param": "2"})))
        monte_carlo.add(feasst.MakeTrialRotate(feasst.args({"weight": "1.", "particle_type": "0", "tunable_param": "50."})))
        monte_carlo.add(feasst.MakeTrialPivot(feasst.args({"weight": "1.", "particle_type": "0", "tunable_param": "50."})))
        monte_carlo.add(feasst.MakeTrialCrankshaft(feasst.args({"weight": "1.", "particle_type": "0", "tunable_param": "50."})))
        monte_carlo.add(feasst.MakeTrialReptate(feasst.args({"weight": "1.", "particle_type": "0", "max_length": "1"})))
        monte_carlo.add(feasst.MakeTrialSwapSites(feasst.args({
            "weight": "1.",
            "particle_type": "0",
            "site_type1": "0",
            "site_type2": "1"
        })))
        monte_carlo.add(feasst.MakeTrialGrowLinear(feasst.MakeTrialComputeMove(), feasst.args({
            "weight": str(1./4./10.),
            "particle_type": "0",
            "num_steps": "4"
        })))
        feasst.add_deprotonation(monte_carlo, feasst.args({
            "reactant_type": "0",
            "reactant_site_type": "0",
            "new_site_type": "1",
            "add_type": "1",
        }))
        
        # trials for counter ions
        monte_carlo.add(feasst.MakeTrialTranslate(feasst.args({"weight": "1.", "particle_type": "1", "tunable_param": "2"})))
        feasst.add_trial_transfer(monte_carlo, feasst.args({"particle_type": "1"}))
        
        # analysis
        steps_per = int(1e3)
        monte_carlo.add(feasst.MakeCheckEnergy(feasst.args({"steps_per": str(steps_per),
                                                            "tolerance": str(1e-10)})))
        monte_carlo.add(feasst.MakeLog(feasst.args({"steps_per": str(steps_per), "file_name": "log.txt",
                                                    "clear_file": "True"})))
        monte_carlo.add(feasst.MakeMovie(feasst.args({"steps_per": str(steps_per), "file_name": "movie.xyz",
                                                      "clear_file": "True"})))
        monte_carlo.add(feasst.MakeTuner(feasst.args({"steps_per": str(steps_per)})))
        monte_carlo.add(feasst.MakeAnalyzeRigidBonds(feasst.args({"steps_per": str(steps_per)})))
        self.assertEqual(monte_carlo.configuration().particle_type_to_group(0), 1)
        monte_carlo.add(feasst.MakeRecenterParticles(feasst.args({"steps_per": str(int(1e2)),
            "group_index": str(monte_carlo.configuration().particle_type_to_group(0))})))
        monte_carlo.add(feasst.MakeCheckPhysicality(feasst.args({"steps_per": str(steps_per)})))
        
        monte_carlo.attempt(int(1e4))

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [10]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestChain2Titration) ... 

CPU times: user 83.3 ms, sys: 3.87 ms, total: 87.2 ms
Wall time: 86.2 ms


ok

----------------------------------------------------------------------
Ran 1 test in 0.084s

OK
